In [1]:
# misc
import pandas as pd
from pandas_profiling import ProfileReport
from pycaret.classification import *
import numpy as np
import re
import time
import math
import os

#loader
from prepare_load import fastDescribe, distincValue

# prepared list of file
path = "dat_filtered/"
df_AllConcat = pd.read_excel(path+'df_AllConcat_kapal_v1.xlsx')

In [2]:
df_AllConcat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7502 entries, 0 to 7501
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tanggal          7502 non-null   object
 1   pelabuhan        7502 non-null   object
 2   kapal_berangkat  7502 non-null   int64 
 3   kapal_tiba       7502 non-null   int64 
 4   penumpang_naik   7502 non-null   int64 
 5   penumpang_turun  7502 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 351.8+ KB


In [3]:
df_AllConcat['pelabuhan'].unique()

array(['MUARA ANGKE', 'MARINA ANCOL', 'UNTUNG JAWA', 'LANCANG', 'PARI',
       'TIDUNG/PAYUNG', 'PRAMUKA/PANGGANG', 'KELAPA/HARAPAN', 'SABIRA'],
      dtype=object)

In [4]:
df_piv = pd.pivot_table(df_AllConcat, values = ['kapal_berangkat', 'kapal_tiba', 'penumpang_naik','penumpang_turun'], index = ['tanggal'], columns=['pelabuhan'])
df_piv

kapal_berangkat                                        \
pelabuhan   KELAPA/HARAPAN LANCANG MARINA ANCOL MUARA ANGKE PARI   
tanggal                                                            
2019-01-01             2.0     3.0          6.0         8.0  5.0   
2019-01-02             1.5     4.0          6.0         8.0  4.0   
2019-01-03             2.5     2.0          4.0         8.0  4.0   
2019-01-04             2.5     4.0         10.0         8.0  3.0   
2019-01-05             2.0     4.0          7.0        10.0  4.0   
...                    ...     ...          ...         ...  ...   
2021-07-27             2.5     6.0          0.0         5.0  3.0   
2021-07-28             2.5     6.0          0.0         5.0  2.0   
2021-07-29             2.0     6.0          0.0         4.0  3.0   
2021-07-30             3.5     6.0          0.0         4.0  2.0   
2021-07-31             2.0     6.0          0.0         5.0  3.0   

                                                                 kapal_tiba  \
pelabuhan  PRAMUKA/PANGGANG SABIRA TIDUNG/PAYUNG UNTUNG JAWA KELAPA/HARAPAN   
tanggal                                                                       
2019-01-01              3.0    NaN           6.0         5.0            3.0   
2019-01-02              2.0    NaN           3.0         6.0            3.0   
2019-01-03              3.0    NaN           4.0         5.0            2.0   
2019-01-04              3.0    NaN           5.0         4.0            2.0   
2019-01-05              3.0    NaN           6.0         7.0            3.0   
...                     ...    ...           ...         ...            ...   
2021-07-27              6.0    1.0           4.0         6.0            2.5   
2021-07-28              5.0    1.0           4.0         6.0            1.5   
2021-07-29              5.0    0.0           4.0         6.0            2.5   
2021-07-30              5.0    1.0           4.0         6.0            2.0   
2021-07-31              5.0    0.0           4.0         6.0            3.0   

            ... penumpang_naik penumpang_turun                       \
pelabuhan   ...    UNTUNG JAWA  KELAPA/HARAPAN LANCANG MARINA ANCOL   
tanggal     ...                                                       
2019-01-01  ...          223.0           205.0    55.0        603.0   
2019-01-02  ...          238.0           123.0    57.0        446.0   
2019-01-03  ...          136.0            44.5    23.0        240.0   
2019-01-04  ...          144.0            61.5    76.0        250.0   
2019-01-05  ...          203.0           116.5   144.0        266.0   
...         ...            ...             ...     ...          ...   
2021-07-27  ...           30.0            25.5    20.0          0.0   
2021-07-28  ...           41.0            18.5    25.0          0.0   
2021-07-29  ...           37.0            41.0    17.0          0.0   
2021-07-30  ...           24.0            35.0    19.0          0.0   
2021-07-31  ...           54.0           147.5    12.0          0.0   

                                                                     \
pelabuhan  MUARA ANGKE   PARI PRAMUKA/PANGGANG SABIRA TIDUNG/PAYUNG   
tanggal                                                               
2019-01-01      1383.0   63.0            131.0    NaN         106.0   
2019-01-02       445.0   48.0            146.0    NaN          92.0   
2019-01-03       243.0  138.0            114.0    NaN         102.0   
2019-01-04       385.0  150.0            141.0    NaN         141.0   
2019-01-05       299.0  167.0            344.0    NaN         377.0   
...                ...    ...              ...    ...           ...   
2021-07-27        50.0   11.0             92.0   22.0           7.0   
2021-07-28        33.0    2.0             50.0    0.0          12.0   
2021-07-29        24.0    9.0             75.0    4.0          16.0   
2021-07-30        52.0    3.0             87.0    0.0          13.0   
2021-07-31        48.0    8.0    

In [5]:
df_piv.columns = df_piv.columns.droplevel(0)
df_piv = df_piv.reset_index().rename_axis(None, axis=1)
df_piv.head()

,tanggal,KELAPA/HARAPAN,LANCANG,MARINA ANCOL,MUARA ANGKE,PARI,PRAMUKA/PANGGANG,SABIRA,TIDUNG/PAYUNG,UNTUNG JAWA,...,UNTUNG JAWA,KELAPA/HARAPAN,LANCANG,MARINA ANCOL,MUARA ANGKE,PARI,PRAMUKA/PANGGANG,SABIRA,TIDUNG/PAYUNG,UNTUNG JAWA
0,2019-01-01,2.0,3.0,6.0,8.0,5.0,3.0,NaN,6.0,5.0,...,223.0,205.0,55.0,603.0,1383.0,63.0,131.0,NaN,106.0,106.0
1,2019-01-02,1.5,4.0,6.0,8.0,4.0,2.0,NaN,3.0,6.0,...,238.0,123.0,57.0,446.0,445.0,48.0,146.0,NaN,92.0,208.0
2,2019-01-03,2.5,2.0,4.0,8.0,4.0,3.0,NaN,4.0,5.0,...,136.0,44.5,23.0,240.0,243.0,138.0,114.0,NaN,102.0,140.0
3,2019-01-04,2.5,4.0,10.0,8.0,3.0,3.0,NaN,5.0,4.0,...,144.0,61.5,76.0,250.0,385.0,150.0,141.0,NaN,141.0,138.0
4,2019-01-05,2.0,4.0,7.0,10.0,4.0,3.0,NaN,6.0,7.0,...,203.0,116.5,144.0,266.0,299.0,167.0,344.0,NaN,377.0,216.0


## The rest Preparation we move into Spreadsheet, you can check here: [Prepare-Process](https://drive.google.com/drive/folders/1jUqd-g3E8CXfAakHo1IqvCKUVMBRDagF?usp=sharing)